# L-Alanin with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up.

In [1]:
from xharpy import shelxl_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions, refine, write_cif, write_res, write_fcf
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [4]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [5]:
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [6]:
hkl = shelxl_hkl2pd('iam.hkl')

In [7]:
constraint_dict = lst2constraint_dict('iam.lst')

In [8]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}

In [9]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [10]:
computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.15,
    'gridinterpolation': 4,
    #'poissonsolver': poissonsolver,d
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    #'kpts': {'size': (2, 2, 2), 'gamma': True},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2
}

In [11]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-08 00:05:12.173687
Preparing
  calculating the core structure factor for O
  calculating the core structure factor for N
  calculating the core structure factor for H
  calculating the core structure factor for C
[[1.99473463 1.9790696  1.95338798 ... 1.33527687 1.33317654 1.32968741]
 [1.99473463 1.9790696  1.95338798 ... 1.33527687 1.33317654 1.32968741]
 [1.99301777 1.97229924 1.93851635 ... 1.19160036 1.18927196 1.18540698]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [1.99030086 1.96164687 1.91532424 ... 1.00949569 1.00700613 1.00287802]]
  calculating first atomic form factors


KeyboardInterrupt: 

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [ ]:
write_res(
    out_res_path=os.path.join(output_folder, 'xharpy_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [ ]:
write_cif(
    output_cif_path=os.path.join(output_folder, 'xharpy.cif'),
    cif_dataset='xharpy',
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [ ]:
hkl